# Predspracovanie dát

Prečo je tento krok dôležitý?
* potrebujeme mať **konzistentné dáta**
    * ak nemáme *zadané hodnoty*, napríklad: šírka=""
    * ak máme *chybné dáta*, napríklad: šírka="-1"
    * ak sa *zmenila logika v dátach*, napríklad ak sme mali najskôr kategórie:['a','b','c'], ale neskôr sme začali používať [0,1,2]
* aby sme mali **kvalitné dáta** na **kvalitné výsledky**
    * napríklad aby sa nám neopakovali dáta - spôsobilo by to skreslenie výsledkov
        
Možností je veľa, vo výsledku sa snažíme o to, aby naše dáta boli *pochopiteľné pre model*. <br>
Ukážeme si základné úpravy, ktoré môžeme vykonať na datasete, aby sme s ním mohli následne pracovať.

Najskôr si ukážeme ako vytvoriť **vlastný dataset** (pre zjednodušenie budeme pracovať s jednoduchou maticou).<br>

* keďže dataset je matica potrebujeme knižnicu, ktorá vie pracovať s maticami a teda použijeme numpy a následne si vytvoríme *ľubovoľnú maticu*

In [1]:
import numpy as np
vlastny_dataset = np.array([[  1.,  5., 10.],
                            [ -2.,  0.,  0.],
                            [  9., 13.,  2.]])
vlastny_dataset

array([[ 1.,  5., 10.],
       [-2.,  0.,  0.],
       [ 9., 13.,  2.]])

Keďže sme na vytvorenie datasetu využili knižnicu NumPy, tak môžeme použiť aj jej metódy. <br>
Napríklad:

In [2]:
vlastny_dataset.shape

(3, 3)

In [3]:
vlastny_dataset.size

9

In [4]:
vlastny_dataset.dtype

dtype('float64')

Tieto a podobné metódy nám často krát pomôžu pri rozsiahlych datasetoch a ich analýze.

## Pri úprave dát máme viaceré techniky, tie delíme do základných kategórií: 
* Imputácia
* Mapovanie kategorických dát
* Škálovanie, Normalizácia a štandardizácia

### 1) Imputácia
* dopĺňa *chýbajúce dáta* a opravuje *poškodené dáta*
* patrí sem: **mean, media a most_frequent**, kde chybné hodnoty nahrádzame:
    * mean: priemernou hodnotou stĺpca z datasetu
    * median: mediánom zo stĺpca datasetu
    * most_frequent: najčastejšou hodnotou stĺpca datasetu
    
* ukážka:
    Máme maticu, ktorá nemá nadefinované niektoré hodnoty:

In [5]:
data =  np.array([[np.nan,      1],
                  [np.nan,      0],
                  [    -3,      2],
                  [     1, np.nan]])
data

array([[nan,  1.],
       [nan,  0.],
       [-3.,  2.],
       [ 1., nan]])

* pri imputácii si môžeme zvoliť stratégiu: *priemerná hodnota*, *medián* (iba pri číselných dátach), *najčastejšia hodnota* (aj pri nečíselných dátach) alebo *konštanta*

* **priemerná hodnota** - aritmetický priemer, ide o súčet všetkých hodnôt vydelený ich počtom

In [6]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
imputed = imputer.fit_transform(data)
data_imputed = imputer.fit_transform(imputed)
data_imputed

array([[-1.,  1.],
       [-1.,  0.],
       [-3.,  2.],
       [ 1.,  1.]])

* **medián** - predstavuje strednú hodnotu, čiže hodnotu, ktorá delí zoradené dáta na dve polovice a to menšie od mediánu a väčšie od mediánu

In [7]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
imputed = imputer.fit_transform(data)
data_imputed = imputer.fit_transform(imputed)
data_imputed

array([[-1.,  1.],
       [-1.,  0.],
       [-3.,  2.],
       [ 1.,  1.]])

* **konštanta** - nemenná veličina/hodnota

In [8]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='constant')
imputed = imputer.fit_transform(data)
data_imputed = imputer.fit_transform(imputed)
data_imputed

array([[ 0.,  1.],
       [ 0.,  0.],
       [-3.,  2.],
       [ 1.,  0.]])

### 2) Mapovanie kategorických dát
* vytvárame mapu, ktorá prepája kategorické názvy s ich číselnými hodnotami

Ukážeme najjednoduchšie mapovanie, kde chceme namapovať druhy jabĺk

In [9]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit_transform(['gala','jonagold','pinova','topaz'])

array([0, 1, 2, 3], dtype=int64)

Teraz si vieme spätne pozrieť, aké vlastnosti sme zakódovali pod čísla 0,1,2,3.

In [10]:
label_encoder.inverse_transform([0, 1, 2, 3])

array(['gala', 'jonagold', 'pinova', 'topaz'], dtype='<U8')

Medzi základné mapovacie techniky patrí aj *OneHotEncoder*, ktorý popíšeme na nasledujúcom príklade.

| Producer/Mark | Color | Engine | Price |
| --- | --- | --- | --- |
| VW Golf | R | electrical | 18450 |
| VW Tiguan | R | diesel | 22560 |
| Ford Focus | G | gasoline | 17600 |
| Renault Clio | G | gasoline | 10999 |
| Honda Civic | - | diesel | 17999 |

Po aplikovaní OHE bude dataset vyzerať nasledujúco:

| Producer/Mark | Color | Electrical | Diesel | Gasoline | Price |
| --- | --- | --- | --- | --- | --- |
| VW Golf | R | 1.0 | 0.0 | 0.0 | 18450 |
| VW Tiguan | R | 0.0 | 1.0 | 0.0 | 22560 |
| Ford Focus | G | 0.0 | 0.0 | 1.0 | 17600 |
| Renault Clio | G | 0.0 | 0.0 | 1.0 | 10999 |
| Honda Civic | - | 0.0 | 1.0 | 0.0 | 17999 |

Z príkladu vidíme, že OneHotEncoder rozdelí prvky podľa kategórii a priradí im binárne hodnoty. Využíva sa v prípadoch, keď labelEncoder nie je dostačujúci alebo priamo nie je vhodný. LabelEncoder automaticky dáva váhu kategóriám keďže im pripisuje celé čísla a navyšuje ich, čomu sa vyhneme používaním 0 a 1 ako pri OnehotEncoder.

### 3) Škálovanie, normalizácia a štandardizácia
#### **Škálovanie**
* škálovanie je potrebné pri datasetoch, ktoré majú rôzne veľké intervaly a ak by sa nevykonalo, tak jednotlivé vlastnosti by nám skresľovali výsledky
* dáta sa dajú na jednu škálu

#### Škálovacie techniky:
Techniky využívame aby sme nemali v datasete vysoké hodnoty v jednom stĺpci a v druhom neporovnateľne nízke, pri výpočtoch by mali vysoké väčšiu váhu a skresľovali výpočty. Snažíme sa teda dať hodnoty na rovnaký pomer a zachovať ich rozptyl.

##### **Štandardizácia**
* používame ju ak porovnávame hodnoty, ktoré sú v rôznych jednotkách alebo ak má náš dataset gaussové rozmiestnenie hodnôt
* škáluje hodnoty tak, aby mali priemernú hodnotu 0 a smerodajnú odchýlku 1
* pre štandardizáciu využívame *StandardScaler*

##### **Normalizácia**
* používa sa vtedy, ak hodnoty v datasete majú rôzne váhy alebo nevieme aké je ich distribúcia
* používa sa minimálna a maximálna hodnota
* škáluje hodnoty do intervalu [0,1] alebo [-1,1] 
* pre normalizáciu sa využíva metóda *MinMaxScaler*

* vytvoríme si jednoduchú maticu, ktorá má *2 vlastnosti a 4 prvky*
* metóda *StandardScaler* transformuje data (viacej o trénovacích a testovacích dátach a učení sa dozviete v kapitole 8) 

In [11]:
data = np.array([[ 1, 1],
                 [ 1, 0],
                 [ 0, 1],
                 [ 0, 0]])
scaler = preprocessing.StandardScaler()
scaler_transform = scaler.fit_transform(data)
scaler_transform

array([[ 1.,  1.],
       [ 1., -1.],
       [-1.,  1.],
       [-1., -1.]])

Vzniknutá *transformovaná matica* má **škálované dáta** ( 1->0.5, 2->1, 0->0 ), dáta si **zachovávajú vzdialenosti** medzi sebou.
* každá vlastnosť má **priemer 0**.

In [12]:
scaler_transform.mean(axis=0)

array([0., 0.])

* a ich **smerodajná odchýlka je 1**, čiže hodnoty sa blížia k strednej hodnote

In [13]:
scaler_transform.std(axis=0)

array([1., 1.])

Ďalší príklad pre štandardizáciu:

In [14]:
data = np.array([[ 10, -100,  2000],
                 [ 20,    0,     0],
                 [  0,  100, -2000]])
standard_scaler = preprocessing.StandardScaler()
standard_scaler_transformed = standard_scaler.fit_transform(data)
standard_scaler_transformed

array([[ 0.        , -1.22474487,  1.22474487],
       [ 1.22474487,  0.        ,  0.        ],
       [-1.22474487,  1.22474487, -1.22474487]])

In [15]:
standard_scaler_transformed.mean(0)

array([0., 0., 0.])

In [16]:
standard_scaler_transformed.std(0)

array([1., 1., 1.])

* dáta sa škálovali a máme pre každú vlastnosť priemernú hodnotu 0 a smerodajnú odchýlku 1

Príklad na normalizáciu:

In [17]:
data = np.array([[ 1, -1,  2],
                 [ 2,  0,  0],
                 [ 0,  1, -2]])
min_max_scaler = preprocessing.MinMaxScaler()

In [18]:
min_max_transformed = min_max_scaler.fit_transform(data)
min_max_transformed

array([[0.5, 0. , 1. ],
       [1. , 0.5, 0.5],
       [0. , 1. , 0. ]])

In [19]:
min_max_scaler.scale_

array([0.5 , 0.5 , 0.25])

#### 1. otázka
**Priraďte príklady k škálovacím technikám (normalizácia a štandardizácia):**
1) Máme dataset, kde je *vek a príjem*. Jedna vlastnosť je v rozmedzí 0-100 a druhá 0-100 000. -ii
2) Dataset obsahuje dáta o *výške a váhe* ľudí. Výška je v cm a váha je v kg. -i <br>
i) štandardizácia <br>
ii) normalizácia

"Správne. Pri prvom datasete musíme normalizovať, inak by mal príjem omnoho väčšiu váhu. Pri druhom datasete máme rôzne jednotky a teda ide o štandardizáciu."

#### 1. programovacia úloha
**Vyskúšajte si spraviť škálovanie na datasete:**<br>
data =  np.array( 
[[  2,  1, -2], 
[  4,  0,  7], 
[ -3,  2,  1], 
[  1, -2,  4]]) <br>
**a overte, či dataset po transformácii spĺňa:** <br>
    - priemerná hodnota jednotlivých vlastností je 0 <br>
    - smerodajná odchýlka vlastností je 1 <br>

In [11]:
# riešenie
from sklearn import preprocessing
import numpy as np

data =  np.array([[  2,  1, -2],
                  [  4,  0,  7],
                  [ -3,  2,  1],
                  [  1, -2,  4]])

scaler = preprocessing.StandardScaler().fit(data)
scaler_transform = scaler.transform(data)
#alebo
# scaler = preprocessing.StandardScaler()
# scaler_transform = scaler.fit_transform(data)

scaler_transform.mean(axis=0)
# scaler_transform.std(axis=0)

array([0., 0., 0.])

In [12]:
standard_scaler=preprocessing.StandardScaler()
standard_scaler_transformed=standard_scaler.fit_transform(data)
standard_scaler_transformed.mean(axis=0)
standard_scaler_transformed.std(axis=0)

array([1., 1., 1.])

#### Materiály na dodatočné štúdium
Stránka, ktorá popisuje tvorbu datasetu a na čo si dávať pozor. <br>
https://blog.cloudfactory.com/steps-to-create-custom-data-sets-for-computer-vision <br>

Dokumentácia k metódam knižnice NumPy. <br>
https://numpy.org/doc/stable/user/quickstart.html <br>

Komplexný rozbor predspracovania dát, popísaný do detailov. <br>
https://bdataanalytics.biomedcentral.com/articles/10.1186/s41044-016-0014-0